# new strategy:
### hard code regex 
1. pull image/ text use OCR textract when neccessary.  Sometimes there are no captions, or detailed descriptions of the image, we can get a weak/general description with Pre-trained image captions

2. With regex, pull brief description, the detailed description of FIG and tie it back to the image figures

3. Save Everything in a table image fig to text 

4. Use the information to find other parts the text that relate to the image with text similarity search that were not referenced by the author 

In [22]:
AWS_ACCESS_KEY_ID=""
AWS_SECRET_ACCESS_KEY=""
AWS_SESSION_TOKEN=""

In [1]:
import boto3
import cv2
import os
import pytesseract
import fitz  # PyMuPDF
import io
import re
import glob
import spacy
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

2023-06-28 16:42:21.707551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# file path you want to extract images from
file = "US9061754.pdf"
# open the file
pdf_file = fitz.open(file)
min_width = 50
min_height = 50
output_dir = "extracted_images"
output_format = "png"

In [24]:
# Iterate over PDF pages
image_paths =[]
for page_index in range(len(pdf_file)):
    # Get the page itself
    page = pdf_file[page_index]
    # Get image list
    image_list = page.get_images(full=True)
    # Print the number of images found on this page
    if image_list:
        print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    else:
        print(f"[!] No images found on page {page_index}")
    # Iterate over the images on the page
    for image_index, img in enumerate(image_list, start=1):
        # Get the XREF of the image
        xref = img[0]
        # Extract the image bytes
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]
        # Get the image extension
        image_ext = base_image["ext"]
        # Load it to PIL
        image = Image.open(io.BytesIO(image_bytes))
        # Check if the image meets the minimum dimensions and save it
        if image.width >= min_width and image.height >= min_height:
            image_path = f"image{page_index + 1}_{image_index}.{output_format}"
            image_paths.append("extracted_images/"+image_path)
            image.save(
                open(os.path.join(output_dir, image_path), "wb"),
                format=output_format.upper())
        else:
            print(f"[-] Skipping image {image_index} on page {page_index} due to its small size.")

[+] Found a total of 1 images in page 0
[+] Found a total of 1 images in page 1
[+] Found a total of 1 images in page 2
[+] Found a total of 1 images in page 3
[+] Found a total of 1 images in page 4
[+] Found a total of 1 images in page 5
[+] Found a total of 1 images in page 6
[+] Found a total of 1 images in page 7
[+] Found a total of 1 images in page 8
[+] Found a total of 1 images in page 9
[+] Found a total of 1 images in page 10
[+] Found a total of 1 images in page 11
[+] Found a total of 1 images in page 12
[+] Found a total of 1 images in page 13
[+] Found a total of 1 images in page 14
[+] Found a total of 1 images in page 15
[+] Found a total of 1 images in page 16
[+] Found a total of 1 images in page 17
[+] Found a total of 1 images in page 18
[+] Found a total of 1 images in page 19
[+] Found a total of 1 images in page 20
[+] Found a total of 1 images in page 21
[+] Found a total of 1 images in page 22
[+] Found a total of 1 images in page 23


In [29]:
figures = []
for image in image_paths:        
    client = boto3.client('textract',region_name='us-east-1',
                      aws_access_key_id=AWS_ACCESS_KEY_ID,
                      aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                      aws_session_token=AWS_SESSION_TOKEN)

    with open(image, 'rb') as image:
        img = bytearray(image.read())


    response = client.detect_document_text(
            Document={'Bytes': img}
        )
       
    figure = [response['Blocks'][i]['Text'] 
                     for i in range(len(response['Blocks'])) 
                     if 'Text' in response['Blocks'][i] 
                     if 'fig' in response['Blocks'][i]['Text'].lower()
                     if len(response['Blocks'][i]['Text'].lower()) < 10
                     if any(char.isdigit() for char in response['Blocks'][i]['Text'])==True]
    print(figure)
    figures.append(list(set(figure)))

# TODO:
# pull filter out

[]
['FIG. 1', 'FIG. 1']
['FIG. 2']
['FIG. 3B', 'FIG. 3A', 'FIG. 3D', 'FIG. 3C']
['FIG. 4B', 'FIG. 4A', 'FIG. 4D', 'FIG. 4C']
['FIG. 5']
[]
['FIG.8', 'FIG.8']
['FIG. 9', 'FIG. 10']
['FIG.11', 'FIG.11']
['FIG.12', 'FIG.12']
['FIG. 13']
['FIG. 14']
['FIG. 15']
['FIG. 16']
['FIG. 17', 'FIG. 18']
[]
[]
[]
[]
[]
['(FIG.5)', '(FIG.5)', '(FIG.5)']
[]
[]


In [36]:
# image_paths

In [35]:
# figures

In [384]:
doc = fitz.open(file)
text = ""
for page in doc:
    text+=page.get_text()
text = ' '.join(text.split())
print(len(text))

14804


In [34]:
# text

In [447]:
# file = "US7725375.pdf"
# doc = fitz.open(file)
# # blocks = [page[4] for index in range(len(doc)) for page in doc[index].get_text("blocks")]   
# blocks = ' '.join([page.get_text() for page in doc])

In [448]:
alt = re.findall(r'DESCRIPTION OF DRAWINGS(.*?)(DETAILED|DESCRIPTION)',text.replace('\n','').replace('.',''))

In [449]:
FIG = re.findall(r'BRIEF DESCRIPTION[A-Z]* OF THE DRAWINGS(.*?)(DETAILED|DESCRIPTION)',text.replace('\n','').replace('.',''))

In [450]:
FIG

[]

In [28]:
finds = []
for fig in figures:
    findings = ' '.join([block for block in blocks if 'FIG. ' +str(fig) in block])
    if findings !=[]:
        finds.append(findings)
    else:
        finds.append('')
            

In [29]:
# finds

In [30]:
df = pd.DataFrame(columns =[])
df['image_paths'] = image_paths
df['figures'] = figures
df['finds'] = finds


In [33]:
# df

In [376]:
# df['finds'][36].split('\n')